In [1]:
import gradio as gr
import requests
from langchain.llms import Ollama

# TMDB API 키
TMDB_KEY = "97f5ea5f12fa2ebcb7d31beb4ad328b0"

# 임베딩 필요없음 (API)
# 모델 키 필요없음
# Ollama 모델 초기화
model = Ollama(model="gemma2", base_url="http://localhost:11434")

# TMDB API 요청 함수
def flim(title):
    url = f"https://api.themoviedb.org/3/search/movie?api_key={TMDB_KEY}&query={title}&language=ko-KR"
    # 주소 뒤 파라미터
    # params = {
    #     'api_key': TMDB_KEY,
    #     'query': title,
    #     'language': 'ko-KR'
    # }

    try:
        response = requests.get(url)
        
        # API 응답 상태 확인
        response.raise_for_status()  # HTTP 에러 발생 시 예외 발생
        
        #
        data = response.json()
                       # 에러를 방지해줍니다.
        results = data.get('results', [])

        # 결과가 없는 경우 처리
        if not results:
            return f"'{title}'에 대한 영화를 찾을 수 없습니다. 다른 제목을 입력해 보세요."

        # 첫 번째 영화 정보 추출
        #
        movie = results[0]
        movie_info = f"""🎬 제목: {movie.get('title', '제목 없음')}
        📅 개봉일: {movie.get('release_date', '개봉일 없음')}
        ⭐ 평점: {movie.get('vote_average', '평점 없음')}/10
        📖 줄거리: {movie.get('overview', '줄거리 정보가 없습니다.') or '줄거리 정보가 없습니다.'}
        """
        return movie_info

    except requests.exceptions.RequestException as e:
        return f"API 요청 중 오류 발생: {str(e)}"
    except IndexError:
        return f"영화 정보를 처리하는 중 오류 발생: 검색 결과가 없습니다."

# 챗봇 함수
def flim_chatbot(message):
    # 1. TMDB로부터 영화 정보 받아오기
    movie_info = flim(message)

    # 영화 정보를 찾지 못한 경우 그대로 반환
    if "찾을 수 없습니다" in movie_info or "오류 발생" in movie_info:
        return movie_info

    # 2. AI 모델에 전달할 "프롬프트 구성과 이해"
    prompt = f"""당신은 친근하고 전문적인 영화 전문가 챗봇입니다. 
다음 영화 정보를 바탕으로 사용자에게 재미있고 유익한 정보를 제공해주세요.

영화 정보:
{movie_info}

사용자 질문: {message}

위 정보를 바탕으로 영화에 대해 친근하고 상세하게 설명해주세요."""

    try:
        # Ollama 모델 호출 
        # Ollama 모델에 전달하여 응답을 생성합니다. (invoke)
        response = model.invoke(prompt)
        return response
    
    except Exception as e:
        return f"챗봇 응답 생성 중 오류 발생: {str(e)}"

# Gradio 인터페이스 생성
iface = gr.Interface(
    # 함수만 받습니다
    fn=flim_chatbot,
    # example
    inputs=gr.Textbox(lines=10, placeholder="영화 제목을 입력하세요 (예: 기생충)"),
    outputs=gr.Textbox(lines=20),
    title="🎬 영화 정보 챗봇",
    description="영화 제목을 입력하면 상세한 정보와 함께 AI가 영화에 대해 설명해드립니다!",
)

# 서버 실행
# if __name__ == "__main__":
iface.launch(server_port=7899, server_name="0.0.0.0")

ModuleNotFoundError: No module named 'gradio'

In [ ]:
iface.close()